In [117]:
#!/usr/bin/env python
import rosbag
import rosbag_pandas

from sklearn import linear_model
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.preprocessing import PolynomialFeatures 

# 1- Loading Data

In [118]:
bag = rosbag.Bag('../bags/posData.bag')

df = rosbag_pandas.bag_to_dataframe('../bags/posData.bag')

/home/abdabo/.local/lib/python3.8/site-packages/rosbag_pandas.py:214: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  bag_info = yaml.load(subprocess.Popen(


In [119]:
df

,bluerov_ground_truth_state__child_frame_id,bluerov_ground_truth_state__pose_pose_position_x,bluerov_ground_truth_state__pose_pose_position_y,bluerov_ground_truth_state__pose_pose_position_z,bluerov_ground_truth_state__pose_pose_orientation_x,bluerov_ground_truth_state__pose_pose_orientation_y,bluerov_ground_truth_state__pose_pose_orientation_z,bluerov_ground_truth_state__pose_pose_orientation_w,bluerov_ground_truth_state__pose_covariance0,bluerov_ground_truth_state__pose_covariance1,...,bluerov_ground_truth_state__twist_covariance26,bluerov_ground_truth_state__twist_covariance27,bluerov_ground_truth_state__twist_covariance28,bluerov_ground_truth_state__twist_covariance29,bluerov_ground_truth_state__twist_covariance30,bluerov_ground_truth_state__twist_covariance31,bluerov_ground_truth_state__twist_covariance32,bluerov_ground_truth_state__twist_covariance33,bluerov_ground_truth_state__twist_covariance34,bluerov_ground_truth_state__twist_covariance35
1970-01-01 02:11:53.980,base_link,0.500000,1.000000,-0.104802,-0.000022,0.000023,0.706825,0.707388,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-01 02:11:54.000,base_link,0.500000,1.000000,-0.104802,-0.000022,0.000023,0.706825,0.707388,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-01 02:11:54.020,base_link,0.500000,1.000000,-0.104802,-0.000022,0.000023,0.706825,0.707388,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-01 02:11:54.040,base_link,0.500000,1.000000,-0.104802,-0.000022,0.000023,0.706825,0.707388,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-01 02:11:54.060,base_link,0.500000,1.000000,-0.104802,-0.000022,0.000023,0.706825,0.707388,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 02:12:50.660,base_link,-0.916025,2.746049,-0.117446,-0.000023,-0.000061,0.926425,-0.376479,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-01 02:12:50.680,base_link,-0.918443,2.748481,-0.116204,-0.000023,-0.000059,0.926425,-0.376479,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-01 02:12:50.700,base_link,-0.920831,2.750882,-0.114974,-0.000023,-0.000057,0.926425,-0.376480,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-01 02:12:50.720,base_link,-0.923189,2.753253,-0.113756,-0.000024,-0.000056,0.926424,-0.376481,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2- Data Preprocessing

In [120]:
df = df[[c for c in df.columns if 'covariance' not in c.lower()]] # removing covariance columns , not important
df = df.drop('bluerov_ground_truth_state__child_frame_id',axis=1) # removing frameid column
df.reset_index(drop=True, inplace=True) # reset index , use numbers instead of dates

In [121]:
'''Changing Columns names'''

new_columns = []
for c in df.columns:
    if 'bluerov_ground_truth_state__pose_pose_' in c:
        new_columns.append(c.split('bluerov_ground_truth_state__pose_pose_')[1])
    elif 'bluerov_ground_truth_state__twist_twist_' in c:
        new_columns.append(c.split('bluerov_ground_truth_state__twist_twist_')[1])
    else:
        continue
        
df.columns = new_columns

In [122]:
# removing duplicates in positions x,y,z
df.drop_duplicates(subset=['position_x', 'position_y','position_z']).reset_index(drop=True, inplace=True)
df

,position_x,position_y,position_z,orientation_x,orientation_y,orientation_z,orientation_w,linear_x,linear_y,linear_z,angular_x,angular_y,angular_z
0,0.500000,1.000000,-0.104802,-0.000022,0.000023,0.706825,0.707388,7.930584e-18,1.388272e-17,-1.013664e-15,1.369024e-14,-1.332074e-15,3.785050e-17
1,0.500000,1.000000,-0.104802,-0.000022,0.000023,0.706825,0.707388,7.929980e-18,1.388022e-17,-1.011393e-15,6.614104e-15,3.381892e-16,3.771498e-17
2,0.500000,1.000000,-0.104802,-0.000022,0.000023,0.706825,0.707388,7.929258e-18,1.387505e-17,-1.014284e-15,-7.909221e-15,2.503492e-15,3.761804e-17
3,0.500000,1.000000,-0.104802,-0.000022,0.000023,0.706825,0.707388,7.928370e-18,1.387959e-17,-1.012021e-15,4.863385e-15,5.341881e-15,3.740752e-17
4,0.500000,1.000000,-0.104802,-0.000022,0.000023,0.706825,0.707388,7.931388e-18,1.387778e-17,-1.014919e-15,-2.900952e-16,-3.491909e-15,3.798126e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2834,-0.916025,2.746049,-0.117446,-0.000023,-0.000061,0.926425,-0.376479,-1.218519e-01,1.225300e-01,6.244597e-02,-1.605628e-04,-9.205941e-05,7.055549e-05
2835,-0.918443,2.748481,-0.116204,-0.000023,-0.000059,0.926425,-0.376479,-1.203089e-01,1.209785e-01,6.184848e-02,-1.547190e-04,-8.870742e-05,6.842673e-05
2836,-0.920831,2.750882,-0.114974,-0.000023,-0.000057,0.926425,-0.376480,-1.187855e-01,1.194466e-01,6.125855e-02,-1.490876e-04,-8.547728e-05,6.636154e-05
2837,-0.923189,2.753253,-0.113756,-0.000024,-0.000056,0.926424,-0.376481,-1.172814e-01,1.179341e-01,6.067609e-02,-1.436607e-04,-8.236454e-05,6.435802e-05


# 3- Splitting Dataset 

In [123]:
# our input will be every state of the robot regarding velocity and orientation
X = df.iloc[0:,4:]
X

,orientation_y,orientation_z,orientation_w,linear_x,linear_y,linear_z,angular_x,angular_y,angular_z
0,0.000023,0.706825,0.707388,7.930584e-18,1.388272e-17,-1.013664e-15,1.369024e-14,-1.332074e-15,3.785050e-17
1,0.000023,0.706825,0.707388,7.929980e-18,1.388022e-17,-1.011393e-15,6.614104e-15,3.381892e-16,3.771498e-17
2,0.000023,0.706825,0.707388,7.929258e-18,1.387505e-17,-1.014284e-15,-7.909221e-15,2.503492e-15,3.761804e-17
3,0.000023,0.706825,0.707388,7.928370e-18,1.387959e-17,-1.012021e-15,4.863385e-15,5.341881e-15,3.740752e-17
4,0.000023,0.706825,0.707388,7.931388e-18,1.387778e-17,-1.014919e-15,-2.900952e-16,-3.491909e-15,3.798126e-17
...,...,...,...,...,...,...,...,...,...
2834,-0.000061,0.926425,-0.376479,-1.218519e-01,1.225300e-01,6.244597e-02,-1.605628e-04,-9.205941e-05,7.055549e-05
2835,-0.000059,0.926425,-0.376479,-1.203089e-01,1.209785e-01,6.184848e-02,-1.547190e-04,-8.870742e-05,6.842673e-05
2836,-0.000057,0.926425,-0.376480,-1.187855e-01,1.194466e-01,6.125855e-02,-1.490876e-04,-8.547728e-05,6.636154e-05
2837,-0.000056,0.926424,-0.376481,-1.172814e-01,1.179341e-01,6.067609e-02,-1.436607e-04,-8.236454e-05,6.435802e-05


In [124]:
# the target output is the position of the robot x,y,z
y = df.iloc[0:,0:3]
y

,position_x,position_y,position_z
0,0.500000,1.000000,-0.104802
1,0.500000,1.000000,-0.104802
2,0.500000,1.000000,-0.104802
3,0.500000,1.000000,-0.104802
4,0.500000,1.000000,-0.104802
...,...,...,...
2834,-0.916025,2.746049,-0.117446
2835,-0.918443,2.748481,-0.116204
2836,-0.920831,2.750882,-0.114974
2837,-0.923189,2.753253,-0.113756


In [125]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

# 4- Building Model

In [126]:
poly = PolynomialFeatures(degree = 2) 

In [127]:
X_poly = poly.fit_transform(X_train) 
poly.fit(X_poly, y_train) 

PolynomialFeatures()

In [128]:
lm = linear_model.LinearRegression() 
lm.fit(X_poly, y_train) 

LinearRegression()

In [129]:
predictions = lm.predict(poly.fit_transform(X_test))

In [130]:
print('model_score = ', metrics.r2_score(y_test, predictions))

model_score =  0.6442090276052018
